In [721]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# Descarga del target
# ==============================================================================
from google.colab import files

#Extra
# ==============================================================================
from datetime import datetime

In [722]:
# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

#Primer DataFrame Encodeado


##Xgboost Regressor

###Entrenamiento

In [723]:
DataFrame2 = pd.read_csv('/content/short_df_Encoded.csv')

In [724]:
DataFrame2.head()

,Unnamed: 0,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [725]:
DataFrame2.drop(columns = {'Unnamed: 0'}, inplace = True)

In [726]:
DataFrame2.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [727]:
DataFrame2Train = DataFrame2.loc[:,DataFrame2.columns != 'Decision']

In [728]:
DataFrame2Test  =  DataFrame2.loc[:,'Decision']

In [729]:
DataFrame2Train.shape, DataFrame2Test.shape, DataFrame2.shape

((16772, 9), (16772,), (16772, 10))

In [730]:
x_train, x_test, y_train, y_test = train_test_split(DataFrame2Train,DataFrame2Test,test_size = 0.2, random_state = 0)

In [731]:
xg_Regressor1 = xgb.XGBRegressor(colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 200, subsample= 0.8)

In [732]:
xg_Regressor1.fit(x_train,y_train)

[15:40:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [733]:
predsRegressor1 = xg_Regressor1.predict(x_test)

In [734]:
value_pred_Regressor1 = pd.DataFrame()
value_pred_Regressor1['pred'] = predsRegressor1
value_pred_Regressor1['pred'].value_counts()

0.880869    117
0.905074     67
0.895266     61
0.148149     42
0.880080     42
           ... 
0.144152      1
0.627397      1
0.584428      1
0.526376      1
0.342099      1
Name: pred, Length: 1931, dtype: int64

In [735]:
rmseRegressor1 = np.sqrt(mean_squared_error(y_test, predsRegressor1))
print("RMSE: %f" % (rmseRegressor1))

RMSE: 0.371824


In [736]:
lossRegressor1 = log_loss(y_test, predsRegressor1)
print(f"Log Loss is {loss}")

Log Loss is 0.4387157682046272


###Test

In [737]:
DataFrameTest = pd.read_csv('/content/DataFrame_test_encoded.csv')

In [738]:
DataFrameTest.head()

,Unnamed: 0,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Opportunity_ID,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0,0.0,0.0,1.0,0.0,0.0,10689,11.905436,0.0,1.0,1.0
1,1,0.0,0.0,1.0,0.0,0.0,10690,11.787392,0.0,1.0,1.0
2,2,0.0,1.0,0.0,0.0,0.0,10691,9.954062,0.0,1.0,1.0
3,3,0.0,1.0,0.0,0.0,0.0,10692,12.338534,1.0,1.0,1.0
4,4,0.0,1.0,0.0,0.0,0.0,10693,8.657390,0.0,0.0,0.0


In [739]:
DataFrameTest.drop(columns = {'Unnamed: 0'}, inplace = True)

In [740]:
DataFrameTest.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Opportunity_ID,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0.0,0.0,1.0,0.0,0.0,10689,11.905436,0.0,1.0,1.0
1,0.0,0.0,1.0,0.0,0.0,10690,11.787392,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,0.0,10691,9.954062,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.0,10692,12.338534,1.0,1.0,1.0
4,0.0,1.0,0.0,0.0,0.0,10693,8.657390,0.0,0.0,0.0


In [741]:
DescargaPredRegressor1 = pd.DataFrame()
DescargaPredRegressor1['Opportunity_ID'] = DataFrameTest['Opportunity_ID']
DescargaPredRegressor1.set_index('Opportunity_ID', inplace = True)
#Tambien separo para el test
#-----------------------------------------------------------------------
DescargaPredClassificator1 = pd.DataFrame()
DescargaPredClassificator1['Opportunity_ID'] = DataFrameTest['Opportunity_ID']
DescargaPredClassificator1.set_index('Opportunity_ID', inplace = True)

In [742]:
DataFrameTest = DataFrameTest.drop(columns={'Opportunity_ID'})
DataFrameTest.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0.0,0.0,1.0,0.0,0.0,11.905436,0.0,1.0,1.0
1,0.0,0.0,1.0,0.0,0.0,11.787392,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,0.0,9.954062,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.0,12.338534,1.0,1.0,1.0
4,0.0,1.0,0.0,0.0,0.0,8.657390,0.0,0.0,0.0


In [743]:
predRegressor_Test1 = xg_Regressor1.predict(DataFrameTest)

In [744]:
DescargaPredRegressor1['target'] = predRegressor_Test1
DescargaPredRegressor1

,target
Opportunity_ID,
10689,0.465789
10690,0.513280
10691,0.621303
10692,0.449422
10693,0.726389
...,...
12364,0.506197
12365,0.569678
12366,0.534197


In [745]:
DescargaPredRegressor1['target'].value_counts()

0.655410    19
0.796289    14
0.478911    12
0.677937    11
0.795427    10
            ..
0.337986     1
0.479113     1
0.271105     1
0.762912     1
0.419750     1
Name: target, Length: 1084, dtype: int64

In [746]:
#csv1 = DescargaPredRegressor1.to_csv('xgboost_regression_pred1.csv')
#files.download('xgboost_regression_pred1.csv')

##Xgboost Clasificator

###Entrenamiento

In [747]:
X_Entrenamiento, X_EntrenamientoTest, Y_TestEntrenamiento, Y_TestTest = train_test_split(DataFrame2Train, DataFrame2Test,test_size = 0.2, random_state = 0)

In [748]:
X_Entrenamiento.shape, X_EntrenamientoTest.shape, Y_TestEntrenamiento.shape, Y_TestTest.shape

((13417, 9), (3355, 9), (13417,), (3355,))

In [749]:
xg_classificator1 = XGBClassifier(colsample_bytree = 0.8, gamma = 1, learning_rate = 0.01, max_depth = 20, min_child_weight = 1, n_estimators = 200, subsample = 0.8)

In [750]:
xg_classificator1.fit(X_Entrenamiento,Y_TestEntrenamiento)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [751]:
predsClassificator1 = xg_classificator1.predict(X_EntrenamientoTest)

In [752]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(Y_TestTest, predsClassificator1))
print(classification_report(Y_TestTest, predsClassificator1))
pd.crosstab(Y_TestTest, predsClassificator1)

0.8113263785394933
              precision    recall  f1-score   support

         0.0       0.82      0.73      0.77      1466
         1.0       0.81      0.88      0.84      1889

    accuracy                           0.81      3355
   macro avg       0.81      0.80      0.81      3355
weighted avg       0.81      0.81      0.81      3355



col_0,0.0,1.0
Decision,,
0.0,1069,397
1.0,236,1653


In [753]:
value_predClassificator1 = pd.DataFrame()
value_predClassificator1['pred'] = predsClassificator1
value_predClassificator1['pred'].value_counts()

1.0    2050
0.0    1305
Name: pred, dtype: int64

In [754]:
rmseClassificator1 = np.sqrt(mean_squared_error(Y_TestTest, predsClassificator1))
print("RMSE: %f" % (rmseClassificator1))
accuracy = accuracy_score(Y_TestTest, predsClassificator1)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

RMSE: 0.434366
Accuracy: 81.13%


In [755]:
lossClassificator1 = log_loss(Y_TestTest, predsClassificator1)
print(f"Log Loss is {loss}")

Log Loss is 0.4387157682046272


###Test

In [756]:
predClassificator_Test1 = xg_classificator1.predict(DataFrameTest)

In [757]:
DescargaPredClassificator1['target'] = predClassificator_Test1
DescargaPredClassificator1

,target
Opportunity_ID,
10689,0.0
10690,0.0
10691,1.0
10692,0.0
10693,1.0
...,...
12364,0.0
12365,1.0
12366,0.0


In [758]:
DescargaPredClassificator1['target'].value_counts()

1.0    939
0.0    628
Name: target, dtype: int64

In [759]:
#csv2 = DescargaPredClassificator1.to_csv('xgboost_clasification_pred1.csv')
#files.download('xgboost_clasification_pred1.csv')

#Segundo DataFrame Encodeado

##Xgboost Regressor

###Entrenamiento

In [760]:
DataFrame3 = pd.read_csv('/content/DataFrame_train_encoded1.csv')

In [761]:
DataFrame3.head()

,Unnamed: 0,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


In [762]:
DataFrame3.drop(columns = {'Unnamed: 0'}, inplace = True)

In [763]:
DataFrame3.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


In [764]:
DataFrame3Train = DataFrame3.loc[:,DataFrame3.columns != 'Decision']

In [765]:
DataFrame3Test  =  DataFrame3.loc[:,'Decision']

In [766]:
DataFrame3Train.shape, DataFrame3Test.shape, DataFrame3.shape

((16772, 13), (16772,), (16772, 14))

In [767]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(DataFrame3Train,DataFrame3Test,test_size = 0.2, random_state = 0)

In [768]:
xg_Regressor2 = xgb.XGBRegressor(colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 200, subsample= 0.8)

In [769]:
xg_Regressor2.fit(x_train2,y_train2)

[15:40:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [770]:
predsRegressor2 = xg_Regressor2.predict(x_test2)

In [771]:
value_pred_Regressor2 = pd.DataFrame()
value_pred_Regressor2['pred'] = predsRegressor2
value_pred_Regressor2['pred'].value_counts()

0.089297    62
0.949236    29
0.914230    26
0.116220    21
0.912759    19
            ..
0.627495     1
0.387968     1
0.372344     1
0.288362     1
0.604299     1
Name: pred, Length: 2455, dtype: int64

In [772]:
rmseRegressor2 = np.sqrt(mean_squared_error(y_test2, predsRegressor2))
print("RMSE: %f" % (rmseRegressor2))

RMSE: 0.361905


In [773]:
lossRegressor2 = log_loss(y_test2, predsRegressor2)
print(f"Log Loss is {loss}")

Log Loss is 0.4387157682046272


###Test

In [774]:
DataFrameTest3 = pd.read_csv('/content/DataFrame_test_encoded1.csv')

In [775]:
DataFrameTest3.head()

,Unnamed: 0,Opportunity_ID,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,2,10689,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,7,10690,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,8,10691,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,14,10692,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,15,10693,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [776]:
DataFrameTest3.drop(columns = {'Unnamed: 0'}, inplace = True)

In [777]:
DataFrameTest3.head()

,Opportunity_ID,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,10689,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,10690,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,10691,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,10692,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,10693,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [778]:
DescargaPredRegressor2 = pd.DataFrame()
DescargaPredRegressor2['Opportunity_ID'] = DataFrameTest3['Opportunity_ID']
DescargaPredRegressor2.set_index('Opportunity_ID', inplace = True)
#Tambien separo para el test
#-----------------------------------------------------------------------
DescargaPredClassificator2 = pd.DataFrame()
DescargaPredClassificator2['Opportunity_ID'] = DataFrameTest3['Opportunity_ID']
DescargaPredClassificator2.set_index('Opportunity_ID', inplace = True)

In [779]:
DataFrameTest3 = DataFrameTest3.drop(columns={'Opportunity_ID'})
DataFrameTest3.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [780]:
predRegressor_Test2 = xg_Regressor2.predict(DataFrameTest3)

In [781]:
DescargaPredRegressor2['target'] = predRegressor_Test2
DescargaPredRegressor2

,target
Opportunity_ID,
10689,0.483180
10690,0.475820
10691,0.615167
10692,0.306613
10693,0.597742
...,...
12364,0.474621
12365,0.562017
12366,0.677410


In [782]:
DescargaPredRegressor2['target'].value_counts()

0.741363    22
0.505418    13
0.411432     9
0.628915     9
0.559316     7
            ..
0.503141     1
0.208792     1
0.551961     1
0.541799     1
0.312732     1
Name: target, Length: 1190, dtype: int64

In [783]:
#csv3 = DescargaPredRegressor1.to_csv('xgboost_regression_pred2.csv')
#files.download('xgboost_regression_pred2.csv')

##Xgboost Clasificator

###Entrenamiento

In [784]:
X_Entrenamiento2, X_EntrenamientoTest2, Y_TestEntrenamiento2, Y_TestTest2 = train_test_split(DataFrame3Train, DataFrame3Test,test_size = 0.2, random_state = 0)

In [785]:
X_Entrenamiento2.shape, X_EntrenamientoTest2.shape, Y_TestEntrenamiento2.shape, Y_TestTest2.shape

((13417, 13), (3355, 13), (13417,), (3355,))

In [786]:
xg_classificator2 = XGBClassifier(colsample_bytree = 0.8, gamma = 1, learning_rate = 0.01, max_depth = 20, min_child_weight = 1, n_estimators = 200, subsample = 0.8)

In [787]:
xg_classificator2.fit(X_Entrenamiento2,Y_TestEntrenamiento2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [788]:
predsClassificator2 = xg_classificator2.predict(X_EntrenamientoTest2)

In [789]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(Y_TestTest2, predsClassificator2))
print(classification_report(Y_TestTest2, predsClassificator2))
pd.crosstab(Y_TestTest2, predsClassificator2)

0.8286140089418778
              precision    recall  f1-score   support

           0       0.83      0.76      0.79      1466
           1       0.82      0.88      0.85      1889

    accuracy                           0.83      3355
   macro avg       0.83      0.82      0.82      3355
weighted avg       0.83      0.83      0.83      3355



col_0,0,1
Decision,,
0,1112,354
1,221,1668


In [790]:
value_predClassificator2 = pd.DataFrame()
value_predClassificator2['pred'] = predsClassificator2
value_predClassificator2['pred'].value_counts()

1    2022
0    1333
Name: pred, dtype: int64

In [791]:
rmseClassificator2 = np.sqrt(mean_squared_error(Y_TestTest2, predsClassificator2))
print("RMSE: %f" % (rmseClassificator2))
accuracy = accuracy_score(Y_TestTest2, predsClassificator2)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

RMSE: 0.413988
Accuracy: 82.86%


In [792]:
lossClassificator2 = log_loss(Y_TestTest2, predsClassificator2)
print(f"Log Loss is {loss}")

Log Loss is 0.4387157682046272


###Test

In [794]:
predClassificator_Test2 = xg_classificator2.predict(DataFrameTest3)

In [795]:
DescargaPredClassificator2['target'] = predClassificator_Test2
DescargaPredClassificator2

,target
Opportunity_ID,
10689,0
10690,0
10691,1
10692,0
10693,1
...,...
12364,1
12365,1
12366,1


In [796]:
DescargaPredClassificator2['target'].value_counts()

0    804
1    763
Name: target, dtype: int64

In [ ]:
#csv4 = DescargaPredClassificator2.to_csv('xgboost_clasification_pred2.csv')
#files.download('xgboost_clasification_pred2.csv')

#GridSearch

In [ ]:
"""
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
"""
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.3, 0.6, 0.8],
        'max_depth': [3, 5, 10, 20],
        'learning_rate' : [0.1, 0.01, 0.001],
        'n_estimators' : [10, 100, 150, 200]
        }


In [ ]:
#xgb = xgb.XGBRegressor(learning_rate=0.02, n_estimators=600, objective='binary:logistic', silent=True, nthread=1)

In [ ]:
"""
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(DataFrame2Train,DataFrame2Test), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(DataFrame2Train, DataFrame2Test)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
"""
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
"""
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(DataFrame2Train,DataFrame2Test), verbose=3 )
grid.fit(DataFrame2Train, DataFrame2Test)
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results-01.csv', index=False)